# Presentation Notebook:

### Notebook to summarize everything covered in previous notebooks

- Introduction: Project Background (Dataset & Objective)
- Python Imports
- Preliminary Analysis (NAs, Imbalanced?, Metadata)
- Variable Analysis (Boxplots)
- Feature Engineering for each row: (mean, std, rounding, ect.)
- Scaling each variable: (StandardScaler, Min_Max, Normalizer)
- Under/Oversampling: (RandomUndersample, SMOTE, SMOTETomek)
- Algorithms pros/cons of each: (Logistic_Regression, KNN, Random_Forest, SVM, LightGBM)
- ROC/PR Curves
- Best Algorithm and Final Results

<hr>

## Introduction: Project Background

<hr>

## Python Imports:

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<hr>

## Preliminary Analysis:

In [2]:
# Import datasets and do distribution plots, find null values, metadata, imbalance 

<hr>

## Feature Analysis:

<hr>

## Too Early Machine Learning Model:

### First we need to seperate the TARGET values from the FEATURES.

In [1]:
# Split to X and y

### Next we need to train_test_split the training data. WHY?
- We want a supervised machine learning model (meaning we want to know the correct answers to judge if it is indeed a good model)
- We can split the data into a train and test set. We'll use 70% of the data for training our model, and 30% of the data for testing our model accuracy.

In [3]:
# train_test_split

### Finally lets plug in a simple LogisticRegression model and see what happens:

In [5]:
# LogisticRegression print classification report

### Analysis:

<hr>

## Feature Engineering:

<hr>

## Feature Scaling:

<hr>

## UnderSampling & OverSampling:

<hr>

## Algorithms:

For Each Algorithm:
- Define classification technique
- Useful Parameters
- Pros / Cons
- Classification Report Analysis

#### Lets use the same train_test_split as we did before and try algorithms again:

### Logistic Regresssion:

In [ ]:
# Run model and Run Classification Report: